# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests # create HTTP requests that will be sent to APIs to retrieve stock market data
import xlsxwriter # save well formatted documents from python scripts
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
stocks.info()
# 5 stocks with dual share structures in s&p500

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Ticker  505 non-null    object
dtypes: object(1)
memory usage: 4.1+ KB


In [4]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

* IEX Cloud data products no longer available as of 31 August 2024
* Use Alpha Vantage real-time financial data API instead (limited number of calls allowed per day for free-tier users - 25)

In [5]:
# free api key from 
# api key from alpha vantage
api_key = '41G44NMW29K8L9Q7'

# define symbol (to be iterated through for loop later on) 
symbol = 'AAPL'
function = 'GLOBAL_QUOTE'

# url for alpha vantage api - returns latest price and volume information of ticker of choice
api_url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}"

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Latest price of each stock - technical data for technical analysis
* Market capitalization for each stock - fundemental data for fundemental analysis


#### (1) Make API call to retrieve technical data - "Price"

In [6]:
# make the API request for latest price of each stock
# response = requests.get(base_url + endpoint)

response = requests.get(api_url) # data already in json format

In [7]:
# check if the request was successful

if response.status_code == 200:
    data01 = response.json()  # parse the JSON response
    print(data01)
else:
    print(f"Error: {response.status_code} - {response.text}")

{'Global Quote': {'01. symbol': 'AAPL', '02. open': '217.5900', '03. high': '218.6300', '04. low': '213.0000', '05. price': '214.2900', '06. volume': '79943254', '07. latest trading day': '2024-06-18', '08. previous close': '216.6700', '09. change': '-2.3800', '10. change percent': '-1.0984%'}}


#### (2) Make another API call to retrieve fundemental data - "MarketCapitalization"

In [8]:
# use the same api_key + symbol with new function to retrieve market cap data
function = 'OVERVIEW'

# new api endpoint - returns market capitalization data 
api_url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}"

In [9]:
response = requests.get(api_url) # data already in json format

In [10]:
# check if the request was successful

if response.status_code == 200:
    data02 = response.json()  # parse the JSON response
    print(data02)
else:
    print(f"Error: {response.status_code} - {response.text}")

{'Symbol': 'AAPL', 'AssetType': 'Common Stock', 'Name': 'Apple Inc', 'Description': "Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.", 'CIK': '320193', 'Exchange': 'NASDAQ', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'TECHNOLOGY', 'Industry': 'ELECTRONIC COMPUTERS', 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US', 'FiscalYearEnd': 'September', 'LatestQuarter': '2024-03-31', 'MarketCapitalization': '3285944107000', 'EBITDA': '129629004000', 'PERatio': '33.33', 'PEGRatio': '2.293', 'BookValue': '4.837', 'D

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [11]:
# define a function to extract the latest price from the API response data
def latest_price(data):
    try:
        # check if 'Global Quote' is not empty
        if data['Global Quote']:
            return float(data['Global Quote']['05. price'])
        else:
            # return NaN if 'Global Quote' is empty
            return np.nan
    except KeyError:
        # handle the case where the expected key is missing
        print("Error: '05. price' key not found in the data.")
        return np.nan

In [12]:
latest_price(data01)

214.29

In [13]:
# customize function to isolate "MarketCapitalization" value from json response
def mkt_cap(data):
    try: 
        # check if data is not empty
        if data: 
            return float(data['MarketCapitalization'])
        else: 
            # return nan if data is empty
            return np.nan
    except KeyError: 
        # handle the case where the expected key is missing 
        print("Error: 'MarketCapitalization' key is not found in the data.")
        return np.nan

In [14]:
mkt_cap(data02)

3285944107000.0

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [15]:
# create dataframe 'df'
columns = ['ticker','latest_price','market_capitalization', 'no_of_shares_to_purchase']
df = pd.DataFrame(columns = columns)
df

,ticker,latest_price,market_capitalization,no_of_shares_to_purchase


In [16]:
type(stocks)

pandas.core.frame.DataFrame

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [17]:
for index, row in stocks.iterrows():
    print(row[0])

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
ALXN
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBY
BDX
BEN
BF.B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK.B
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COG
COO
COP
COST
COTY
CPB
CPRT
CRM
CSCO
CSX
CTAS
CTL
CTSH
CTVA
CTXS
CVS
CVX
CXO
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQIX
EQR
ES
ESS
ETFC
ETN
ETR
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDX
FE
FFIV
FIS
FISV
FITB
FLIR
FLS
FLT
FMC
FOX
FOXA
FRC
FRT
FTI
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HBI
HCA
HD
HES
HFC
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRB
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILM

In [18]:
# populate table 'latest_price' with values called through API
# row-wise iteration in pandas
for index, row in stocks.iterrows():
    # define parameters
    function = 'GLOBAL_QUOTE'
    symbol = row['Ticker']
    api_key = '41G44NMW29K8L9Q7'
    
    api_url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}"
    
    # set the ticker column using .at[]
    df.at[index, 'ticker'] = symbol
    
    # call API and send get request 
    response = requests.get(api_url) 
    # parse the json response
    if response.status_code == 200:
        data = response.json()  # parse the JSON response
        # print(data)
    else:
        print(f"Error: {response.status_code} - {response.text}") # why is the error message not printed
        
    # extract the latest price and populate the 'latest_price' column
    if data: 
        df.at[index, 'latest_price'] = latest_price(data)
    else:
        # if there's no valid data, set the latest_price to NaN
        df.at[index, 'latest_price'] = np.nan
        
return df      

In [22]:
# populate table 'mkt_cap' with values called through API
# row-wise iteration in pandas
for index, row in stocks.iterrows():
    # define parameters
    function = 'OVERVIEW'
    symbol = row['Ticker']
    api_key = '41G44NMW29K8L9Q7'
    
    api_url = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&apikey={api_key}"
    
    # call API and send get request 
    response = requests.get(api_url) 
    # parse the json response
    if response.status_code == 200:
        data = response.json()  # parse the JSON response
        # print(data)
    else:
        print(f"Error: {response.status_code} - {response.text}")
        
    # extract the latest price and populate the 'market_capitalization' column
    if data: 
        df.at[index, 'market_capitalization'] = mkt_cap(data)
    else:
        # if there's no valid data, set the latest_price to NaN
        df.at[index, 'market_capitalization'] = np.nan
                           

In [23]:
df

,ticker,latest_price,market_capitalization,no_of_shares_to_purchase
0,A,134.9,39358554000.0,NaN
1,AAL,11.28,7400695000.0,NaN
2,AAP,64.28,3832521000.0,NaN
3,AAPL,214.29,3285944107000.0,NaN
4,ABBV,171.36,304594911000.0,NaN
...,...,...,...,...
500,YUM,135.42,38561055000.0,NaN
501,ZBH,106.69,21870942000.0,NaN
502,ZBRA,295.25,15181578000.0,NaN
503,ZION,41.4,6092246000.0,NaN


## Using Batch API Calls to Improve Efficiency

Batch API calls are one of the easiest ways to improve the performance of your code, in terms of efficiency.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 6 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

As for Alpha Vantage, the provider limits their batch API calls to 100 per request, and 25 API calls per day. For free-tier users, retrieving the data in batches of 100 will cut down the number of API calls to be made per day. This makes it possible for all 'price' and 'market cap' data for each stock in S&P500 to be retrieved. 

In [24]:
def chunks(lst, n): # function chunks will take in 2 parameters, the list to be split into chunks and the desired chunk size
    """yield successive n-sized chunks from list"""
    for i in range(0, len(lst), n): # loop starts at 0, increases by n each time, until it reaches the length of the list
        yield lst[i:i + n]

In [25]:
ticker_chunks = list(chunks(stocks['Ticker'], 100))
ticker_strings = [] # list of 6 ticker_strings (100, 100, 100, 100, 100, 5)
for i in range(0, len(ticker_chunks)):
    ticker_strings.append(','.join(ticker_chunks[i]))
    print(ticker_strings[i])

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [34]:
# create new dataframe with 505 rows and the specified columns, initialized with NaN
columns_02 = ['Ticker','Price','Market Capitalization','Number Of Shares to Buy']
df_batch = pd.DataFrame(np.nan, index=range(505), columns=columns_02)

In [35]:
# check if dataframe initialized successfully
df_batch

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
500,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN
502,NaN,NaN,NaN,NaN
503,NaN,NaN,NaN,NaN


In [39]:
# loop through tickers in list of stocks, to populate 'ticker', 'price' and 'market capitalization' columns
for ticker_str in ticker_strings:
    # set the parameters
    
    batch_api_call_url = f"https://www.alphavantage.co/query?function={function}&symbol={ticker_str}&apikey={api_key}"
    data = requests.get(batch_api_call_url).json()
    print(ticker_str)
    print(data)
    
    for ticker in ticker_str.split(','): # iterate through each ticker and insert data into the DataFrame
        if ticker in data:  # ensure the ticker exists in the returned data
            ticker_data = data[ticker]['Global Quote']

            # get the current length of the DataFrame
            index = len(df_batch)

            # insert new rows with .loc
            df_batch.loc[index] = [
                ticker,
                ticker_data.get('05. price'),
                np.nan,
                np.nan]  
            
df_batch

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
{}
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
{}
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,I

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
500,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN
502,NaN,NaN,NaN,NaN
503,NaN,NaN,NaN,NaN


Alpha Vantage does not allow consumers to fetch data for multiple tickers at a time. Need to make individual requests for each ticker in stocks. Hence you need to subscribe to get a premium API key. Feel free to use mine - available from 19Jun24, through to 19Jul24.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [40]:
df

,ticker,latest_price,market_capitalization,no_of_shares_to_purchase
0,A,134.9,39358554000.0,NaN
1,AAL,11.28,7400695000.0,NaN
2,AAP,64.28,3832521000.0,NaN
3,AAPL,214.29,3285944107000.0,NaN
4,ABBV,171.36,304594911000.0,NaN
...,...,...,...,...
500,YUM,135.42,38561055000.0,NaN
501,ZBH,106.69,21870942000.0,NaN
502,ZBRA,295.25,15181578000.0,NaN
503,ZION,41.4,6092246000.0,NaN


In [44]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio: 1000000


In [45]:
position_size = float(portfolio_size) / len(df.index)

# programmatically loop through each ticker in the pandas dataframe
for i in range(0, len(df['ticker'])-1):
    if not pd.isna(df['latest_price'][i]):  # check if the price is not nan
        # round down because some brokerages do not allow purchase of fractional shares
        df.loc[i, 'no_of_shares_to_purchase'] = math.floor(position_size / df['latest_price'][i]) 
    else:
        df.loc[i, 'no_of_shares_to_purchase'] = np.nan  # set to nan for clarity
df

,ticker,latest_price,market_capitalization,no_of_shares_to_purchase
0,A,134.9,39358554000.0,14
1,AAL,11.28,7400695000.0,175
2,AAP,64.28,3832521000.0,30
3,AAPL,214.29,3285944107000.0,9
4,ABBV,171.36,304594911000.0,11
...,...,...,...,...
500,YUM,135.42,38561055000.0,14
501,ZBH,106.69,21870942000.0,18
502,ZBRA,295.25,15181578000.0,6
503,ZION,41.4,6092246000.0,47


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [46]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter') # initializing the writer object 
df.to_excel(writer,
            sheet_name = 'Recommended Trades',
            index = False) # write data from pandas dataframe to excel file

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [47]:
background_color = '#87CEEB' # skyblue
font_color = '#00008B' # darkblue

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [48]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)

# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [49]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [51]:
writer.close()